### Fine tune OPT model using LORA and int8 on SageMaker

In [ ]:
! pip install -U sagemaker

In [ ]:
! pip install datasets

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

In [ ]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

### Create and upload data

In [ ]:
from datasets import load_dataset

hc3 = load_dataset("Hello-SimpleAI/HC3","all")
hc3["train"].to_json(f"data/all.jsonl")

In [ ]:
import pandas as pd
import json
  
# Opening JSON file
f1 = open('data/all.jsonl')
Lines1 = f1.readlines()

questions, answers = [], []
print(len(Lines1))
for line in Lines1[:10000]:
    row = json.loads(line)
    for answer in row["human_answers"]:
        questions.append(row["question"])
        answers.append(answer)
    for answer in row["chatgpt_answers"]:
        questions.append(row["question"])
        answers.append(answer)

test_file = open("data/test.jsonl","w")
for line in Lines1[10000:]:
    test_file.write(line)

test_file.close()

df = pd.DataFrame()
df["question"] = questions
df["answer"] = answers
df = df.sample(frac = 1)
print(len(df.index))
df_train = df.iloc[:25000,:]
df_val = df.iloc[25000:,:]
print(len(df_train.index))
print(len(df_val.index))
df_train.to_csv("data/train.csv", index=False)
df_val.to_csv("data/val.csv", index=False)

In [ ]:
train_data_url = sess.upload_data(
    path="data/train.csv",
    key_prefix="hc3",
)

valid_data_url = sess.upload_data(
    path="data/val.csv",
    key_prefix="hc3",
)

In [ ]:
print(f"training file path {train_data_url}")
print(f"validation file path {valid_data_url}")

In [ ]:
SM_DATA_DIR = "/opt/ml/input/data"

In [ ]:
hyperparameters = {}

hyperparameters["model_name_or_path"] = "facebook/opt-13b"
hyperparameters["model_dir"] = f"/opt/ml/model"
hyperparameters["train_file"] = f"{SM_DATA_DIR}/train/train.csv"
hyperparameters["validation_file"] = f"{SM_DATA_DIR}/valid/val.csv"
hyperparameters["per_device_train_batch_size"] = 1
hyperparameters["per_device_eval_batch_size"] = 1
hyperparameters["block_size"] = 2048
hyperparameters["num_train_epochs"] = 1

In [ ]:

base_job_name = "opt-instruction-tuning-int8-lora"
estimator = PyTorch(
    base_job_name=base_job_name,
    source_dir="./scripts",
    entry_point="train_int8.py",
    role=role,
    framework_version="1.13.1",
    py_version="py39",
    instance_count=1,
    instance_type="ml.p4d.24xlarge",
    hyperparameters=hyperparameters,
    disable_profiler=True,
    keep_alive_period_in_seconds=300,
    distribution={"torch_distributed": {"enabled": True}},

)

In [ ]:
estimator.fit({"train":train_data_url,"valid":valid_data_url})

In [ ]:
sess.update_training_job(estimator.latest_training_job.job_name, resource_config={"KeepAlivePeriodInSeconds":0})